# Generalized Linear Model (GLM)

------------------------------------

Links for understand topic:
* Stanford CS299: https://www.youtube.com/watch?v=iZTeva0WSTQ&list=PLoROMvodv4rMiGQp3WXShtMGgzqpfVfbU&index=4

The main idea of knowing and using **Generalized Linear Models (GLMs)** is that we are able to solve many classic machine learning problems using basic linear regression principles.

As I understand it, we train a linear regression model and then use a **link function** to transform its output so that it matches the **distribution of the target variable**.

This topic shows us how functions we already know, such as sigmoid, softmax, and others, were created and why they are used.

That is why this topic should be one of **the core theorems** (or core topics) in classical machine learning.

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

np.random.seed(42)

## Optimize parameters by Gradient Ascent

In [63]:
def grad_update(X, y, mu):
    return X.T @ (y - mu) # That's all!

# X.T @ theta => eta => mu

# 1) Linear Regression | mu = eta = X @ theta
def linear_mu(X, theta):
    mu = X @ theta
    return mu

# 2) Logistic regression | mu = 1/(1+exp(-eta)) = 1/(1+exp(-X @ theta))
def logistic_mu(X, theta):
    eta = X @ theta
    eta = np.clip(eta, -500, 500)
    mu = 1/(1+np.exp(-(eta)))
    return mu

# 3) Softmax regression | mu_i = (exp(X @ theta_i)) / sum(exp(X @ theta_j))
# y.shape == mu.shape == (n_samples, n_classes) - Main condition
def softmax_mu(X, theta):
    logits = X @ theta
    logits -= np.max(logits, axis=1, keepdims=True)  # стабилизация
    # logits = np.hstack([logits, np.zeros((X.shape[0], 1))])
    exp_logits = np.exp(logits)
    mu = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
    return mu

In [64]:
# Common stuctured def for optimize parameters
def fit(X, theta_init, y, task, lr=1e-3, n_iters=1000):
    theta = theta_init.copy()
    
    for _ in range(n_iters):
        if task == 'linear':
            mu = linear_mu(X, theta)
        elif task == 'logistic':
            mu = logistic_mu(X, theta)
        elif task == 'softmax':
            mu = softmax_mu(X, theta)
            # y must be one-hot here!
        else:
            raise ValueError("Unknown task")
        
        theta += lr * grad_update(X, y, mu)
        
        # if task == 'softmax':
        #     theta[:, 0] = 0
    
    return theta     

In [65]:
# Generate X (data)
# y = X @ Theta + epsilone

def make_X(n_samples=1000, n_features=3, bias=True, seed=0):
    rng = np.random.default_rng(seed)
    X = rng.normal(size=(n_samples, n_features))
    if bias:
        X = np.c_[np.ones(n_samples), X]
    return X

### Linear Regression problem:

In [66]:
# Linear Regression by GLM:
def generate_linear_data(X, theta, sigma=1.0, seed=0):
    rng = np.random.default_rng(seed)
    mu = X @ theta
    y = mu + rng.normal(scale=sigma, size=mu.shape)
    return y

# Generate data for Linear Regression problem:
X = make_X()
theta_true = [3.2678, 1.365, 0.653, -0.0698]
y_true = generate_linear_data(X, theta_true)

# Let's go use GLM!!!
theta_0 = [1, 1, 1, 1]
theta_train = fit(X, theta_0, y_true, 'linear')
print(f"Theta (TRUE) = {theta_true}")
print(f"Theta (AFTER TRAIN) = {theta_train}")

Theta (TRUE) = [3.2678, 1.365, 0.653, -0.0698]
Theta (AFTER TRAIN) = [ 3.21293235  1.30052909  0.68080355 -0.05252388]


### Logistic Regression problem:

In [67]:
# Logistic Regression by GLM:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def generate_logistic_data(X, theta, seed=0):
    rng = np.random.default_rng(seed)
    eta = X @ theta
    mu = sigmoid(eta)
    y = rng.binomial(1, mu)
    return y

# Generate data for Logistic Regression problem:
X = make_X()
theta_true = [3.2678, 1.365, 0.653, -0.0698]
y_true = generate_logistic_data(X, theta_true)

# Let's go use GLM!!!
theta_0 = [1, 1, 1, 1]
theta_train = fit(X, theta_0, y_true, 'logistic', n_iters=2000)
print(f"Theta (TRUE) = {theta_true}")
print(f"Theta (AFTER TRAIN) = {theta_train}")

# I understand that theta_true != theta_train is normal thing. For solve this I need to generate more data and use regulirization.

Theta (TRUE) = [3.2678, 1.365, 0.653, -0.0698]
Theta (AFTER TRAIN) = [3.1325853  1.38198022 0.52704664 0.11608256]


### Softmax Regression problem:

In [ ]:
# Softmax Regression by GLM:
def softmax(z):
    z = z - np.max(z, axis=1, keepdims=True)
    exp_z = np.exp(z)
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def generate_softmax_data(X, theta, seed=0):
    """
    theta: (n_features, n_classes)
    """
    rng = np.random.default_rng(seed)
    logits = X @ theta
    mu = softmax(logits)

    y = np.array([
        rng.choice(mu.shape[1], p=mu[i])
        for i in range(mu.shape[0])
    ])

    # one-hot
    y_onehot = np.eye(mu.shape[1])[y]
    return y, y_onehot

# Generate data for Softmax Regression problem:
X = make_X()
theta_true = np.array([
    [ 3.0, -1.0,  0.5],
    [ 1.2,  0.3, -0.8],
    [ 0.7, -0.4,  0.2],
    [-0.1,  0.6, -0.3],
]) 
y_true, y_true_onehot = generate_softmax_data(X, theta_true)


# Let's go use GLM!!!
theta_0 = np.array([
    [ 1.0, 1.0, 1.0 ],
    [ 1.0, 1.0, 1.0 ],
    [ 1.0, 1.0, 1.0 ],
    [ 1.0, 1.0, 1.0 ],
]) 
theta_train = fit(X, theta_0, y_true_onehot, 'softmax', n_iters=1000)
print(f"Theta (TRUE) = \n{theta_true}\n")
print(f"Theta (AFTER TRAIN) = \n{theta_train}")

Theta (TRUE) = 
[[ 3.  -1.   0.5]
 [ 1.2  0.3 -0.8]
 [ 0.7 -0.4  0.2]
 [-0.1  0.6 -0.3]]

Theta (AFTER TRAIN) = 
[[ 3.04584903e+00 -7.28946582e-01  6.83097551e-01]
 [ 1.99526300e+00  1.00735078e+00 -2.61377540e-03]
 [ 1.44649430e+00  4.79942781e-01  1.07356292e+00]
 [ 9.05299805e-01  1.56767055e+00  5.27029648e-01]]


Today we talked about **how use GLM**, and if you study basic theory about it you understand that it is **cool thing** and **where we get all this formulations**! **In conclusion**, maybe someone have questions *"HOW?"* or *"WHY?"*, and **that's OK!** Because We did something at first time and did't know nothing.**However in one day** you will be **master** in it! **Keep moving!** 